# Project Object Oriented Programming (Python) - Univelcity Bootcamp

### Name of student: Okonkwo Obinna Uzochukwu

### Project 4.1

We need to design an implement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

In [1]:
import numbers
from datetime import timedelta, datetime
import itertools
from collections import namedtuple

Confirmation = namedtuple('Confirmation', 'account_num transaction_code transaction_id time_utc time')

class Timezone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty')
            
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer')
            
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minute offset must be an integer')
        
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minute ofset must between -59 and 59 (inclusive)')
            
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
         
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset
        

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name


    def __eq__(self, other):
        return (isinstance(other, Timezone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and 
                self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return (f"Timezone(name='{self.name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes}  )")


In [2]:
class Account:
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5
    
    _transaction_codes = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self, account_number, first_name, last_name, timezone=None, initial_balance=0):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        
        if timezone == None:
            timezone = Timezone('UTC', 0, 0)
        self.timezone = timezone
        self._balance = float(initial_balance)
        
        
    @property
    def account_number(self):
        return self._account_number
    
    
    
    @property
    def balance(self):
        return self._balance
    
    
    @property
    def first_name(self):
        return self._first_name
    
    
    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name('_first_name', value, 'First Name')
        
        
    @property   
    def last_name(self):
        return self._last_name
           
    
    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name('_last_name', value, 'Last Name')
        
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
        
    @property   
    def timezone(self):
        return self._timezone
    
    
    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, Timezone):
            raise ValueError('Time Zone must be a valid Time Zone object')
        self._timezone = value
        
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a Real Number')
            
        if value < 0:
            raise ValueError('Interest rate cannot be neagtive')
        cls._interest_rate = value
     
    
    def generate_confirmation_code(self, transaction_code):
        dt_str = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}'
    
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_timezone=None):
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
            
        transaction_code, account_num, raw_dt_utc, transaction_id = parts
        
        try:
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime') from ex
        if preferred_timezone is None:
            preferred_timezone = Timezone('UTC', 0, 0)
            
        if not isinstance(preferred_timezone, Timezone):
            raise ValueError('Invalid Timezone specified')
            
        dt_preferred = dt_utc + preferred_timezone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_timezone.name})"
        
        return Confirmation(account_num, transaction_code, transaction_id,
                            dt_utc.isoformat(), dt_preferred_str)
    
    
    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty')
        setattr(self, attr_name, value)
        
        
        
    @staticmethod    
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Value must be a real number')
            
        if min_value is not None and value < min_value:
            raise ValueError(f'Value must be at least {min_value}.')
            
        return value
        
        
    def make_transaction(self):
        return self.generate_confirmation_code('dummy')

    def deposit(self, value):
        value = Account.validate_real_number(value, 0.01)
            
        transaction_code = Account._transaction_codes['deposit']
        conf_code = self.generate_confirmation_code(transaction_code)
        
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        value = Account.validate_real_number(value, 0.01)
        
        accepted = False
        if self.balance - value < 0:
            transaction_code = Account._transaction_codes['rejected']
        else:
            accepted = True
            transaction_code = Account._transaction_codes['withdraw']
            
        conf_code = self.generate_confirmation_code(transaction_code)
        
        if accepted:
            self._balance -= value
            
        return conf_code
    
    
    def pay_interest(self):
        interest = self.balance * (Account.get_interest_rate() / 100)
        conf_code = self.generate_confirmation_code(Account._transaction_codes['interest'])
        self._balance += interest
        return conf_code

In [3]:
a1 = Account('C200', 'Arinze', 'Okonkwo', initial_balance=100)

In [4]:
vars(a1)

{'_account_number': 'C200',
 '_first_name': 'Arinze',
 '_last_name': 'Okonkwo',
 '_timezone': Timezone(name='UTC', offset_hours=0, offset_minutes=0  ),
 '_balance': 100.0}

In [5]:
a1.full_name

'Arinze Okonkwo'

In [6]:
try:
    a1.withdraw(-19)
except ValueError as ex:
    print(ex)

Value must be at least 0.01.


In [7]:
Account.parse_confirmation_code(a1.make_transaction())

Confirmation(account_num='C200', transaction_code='dummy', transaction_id='100', time_utc='2022-02-09T18:45:20', time='2022-02-09 18:45:20 (UTC)')

In [8]:
a1.deposit(100)

'D-C200-20220209184520-101'

In [9]:
a1.balance

200.0

In [10]:
a1.withdraw(150)

'W-C200-20220209184520-102'

In [11]:
a1.withdraw(160)

'X-C200-20220209184520-103'

In [12]:
a1.balance

50.0